In [1]:
import numpy as np
import cv2
import os, glob
import pandas as pd
from itertools import product
import affine
from affine import Affine
import pyproj
import dask.dataframe as dd
import dask.array as da
import matplotlib.pyplot as plt
import rasterio
import rasterio.features
import keplergl
import ast
import configparser


In [2]:
paths = configparser.ConfigParser()
paths.read_file(open(r'config.txt'))
path = paths.get('paths', 'sat')

In [3]:
pixel_transform = affine.Affine(  # Converte X, Y redimensionado para X, Y real
        -20,
        # largura de 1 pixel em metros /norte-sul/ corresponde a variação na direção Y para fazer a escala para
        # medidas geoespaciais
        0, # rotação Y, como é uma foto a rotação é 0
        7988882.041015626,  # coordenadas geoespaciais Y do pixel do canto superior esquerdo / translação do Y para executar a
        # escala de maneira precisa. A translação é uma transformação na qual todos os pontos em uma linha
        # denominada(nesse caso Y) permanecem estáticos enquanto os outros pontos se movem a uma distância
        # proporcional a distância perpendicular ao ponto de referência estático. Essa translação não altera a área.
         0, # rotação X, foto = rotação 0
        20,  # altura de 1 pixel /leste-oeste / variação na direção X para fazer a escala para medidas
        # geoespaciais
       441509.58984375
  # coordenadas geoespaciais X do pixel do canto superior direito / translação
    )


In [4]:
print(path)

C:\Users\pcosm\Desktop\Teia\S2-Kivu


In [5]:
print(pixel_transform)
pd.set_option("display.precision", 15)

|-20.00, 0.00, 7988882.04|
| 0.00, 20.00, 441509.59|
| 0.00, 0.00, 1.00|


In [6]:
from pyproj import Transformer
transformer = Transformer.from_crs(32723, 4326,  always_xy =True)

In [7]:
img_list = glob.glob(os.path.join(path , '*.tif'))

In [8]:
for p in img_list:
    p = p.split('\\')
    print(p)
    p = p[-1]
    print(p)

['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B11_20m_Mask_NDWI.tif']
B11_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B12_20m_Mask_NDWI.tif']
B12_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B2_20m_Mask_NDWI.tif']
B2_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B3_20m_Mask_NDWI.tif']
B3_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B4_20m_Mask_NDWI.tif']
B4_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B5_20m_Mask_NDWI.tif']
B5_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B6_20m_Mask_NDWI.tif']
B6_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B7_20m_Mask_NDWI.tif']
B7_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B8A_20m_Mask_NDWI.tif']
B8A_20m_Mask_NDWI.tif
['C:', 'Users', 'pcosm', 'Desktop', 'Teia', 'S2-Kivu', 'B8_20m_Mask_NDWI.tif']
B8_20m_Mask_NDW

In [9]:
df_list = []
coords_list = []
cart_list = []
def points():
    for img in img_list:
    # im = cv2.imread(img_list[0], -1)

        im = cv2.imread(img, -1)
        height = range(im.shape[0])
        width = range(im.shape[1])
        im = im.flatten()
        df = pd.DataFrame(im)
        df.columns = ['value']
        df_list.append(df)

        #     height, width = im.shape[:2]

        #     center = (width/2, height/2)

        #     print(width, height)

        #     rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=90, scale=1)

        #     rotated_image = cv2.warpAffine(src=im, M=rotate_matrix, dsize=(width, height))
        coords = np.array(list(product(height, width)))

        #     y = range(rotated_image.shape[0])
        #     x = range(rotated_image.shape[1])
        #     coords = np.array(list(product(y, x)))
        #     print(coords)


        coords_df = pd.DataFrame(coords, columns = ['Y', 'X'])


        yy, xx = pixel_transform * (coords_df['Y'], coords_df['X'])





        #     utm23s = pyproj.CRS('epsg:32723')  # EPSG 32723 (X, Y geoespaciais)
        #     wgs84 = pyproj.CRS('epsg:4326')  # EPSG WGS84 (lat/lng) 4326

        #     latlng_proj = pyproj.Transformer.from_crs(wgs84, utm23s, always_xy=True)  # converte 4326(lat, long) para 32723(X,Y)
        #     proj_latlng = pyproj.Transformer.from_crs(utm23s, wgs84)  # converte 32723(X,Y) para 4326(lat, long)


        #     ul_x, ul_y = latlng_proj.transform(-45.553001496814865, -18.188492870464525)

        xx_latlng, yy_latlng = transformer.transform(xx,yy)

        #     yy_latlng, xx_latlng = proj_latlng.transform(xx, yy)

        xx_latlng = pd.Series(xx_latlng)
        yy_latlng = pd.Series(yy_latlng)

        xx_yy = pd.concat([xx_latlng, yy_latlng], axis=1)
        
        xx_yy.columns = ['Longitude', 'Latitude']
        
        name = img.split('\\')
        name = name[-1]
        name = name.replace('.tif', '')
        
        xx_yy.name = name
        df.name = name
        print(xx_yy.name)
        
        
        

        cart_list.append(coords_df)

        coords_list.append(xx_yy)
    print(len(coords_list))
points()

B11_20m_Mask_NDWI
B12_20m_Mask_NDWI
B2_20m_Mask_NDWI
B3_20m_Mask_NDWI
B4_20m_Mask_NDWI
B5_20m_Mask_NDWI
B6_20m_Mask_NDWI
B7_20m_Mask_NDWI
B8A_20m_Mask_NDWI
B8_20m_Mask_NDWI
10


In [10]:
minimal = df_list[0]['value'].min()
path_namer = path
df_namer = glob.glob(os.path.join(path_namer , '*.tif'))
print(df_namer)

['C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B11_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B12_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B2_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B3_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B4_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B5_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B6_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B7_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B8A_20m_Mask_NDWI.tif', 'C:\\Users\\pcosm\\Desktop\\Teia\\S2-Kivu\\B8_20m_Mask_NDWI.tif']


In [11]:
for name in df_namer:
    print(name)

C:\Users\pcosm\Desktop\Teia\S2-Kivu\B11_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B12_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B2_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B3_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B4_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B5_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B6_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B7_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B8A_20m_Mask_NDWI.tif
C:\Users\pcosm\Desktop\Teia\S2-Kivu\B8_20m_Mask_NDWI.tif


In [12]:
print(df_list[3].name)

B3_20m_Mask_NDWI


In [13]:
points = []
for d in df_list:
    print(d.name)
    merge = pd.concat([d, coords_list[0]], axis=1)
    merge_mask = merge[(merge['value'] > minimal)]
    img_band = df_namer[0].replace(path_namer, '')
    img_band = img_band.replace('_20m_Mask_NDWI.tif', '')
    img_band = img_band.replace('\\', '')
    merge_mask.name = img_band
    print(merge_mask.name)
    df_namer.pop(0)
    coords_list.pop(0)
    #     img_band = df_namer[0].replace(path_namer, '')
    #     img_band = img_band.replace('_20m_Mask_NDWI.tif', '')
    #     merge.name = img_band
    #coords_list.pop(0)
    #    df_namer.pop(0)
    points.append(merge_mask)
    

B11_20m_Mask_NDWI
B11
B12_20m_Mask_NDWI
B12
B2_20m_Mask_NDWI
B2
B3_20m_Mask_NDWI
B3
B4_20m_Mask_NDWI
B4
B5_20m_Mask_NDWI
B5
B6_20m_Mask_NDWI
B6
B7_20m_Mask_NDWI
B7
B8A_20m_Mask_NDWI
B8A
B8_20m_Mask_NDWI
B8


In [14]:
#path_namer = 'C:\\Users\\pcosm\\Desktop\\S2-KIVU\\'


In [15]:
#df_namer = glob.glob(os.path.join(path_namer , '*.tif'))

In [16]:
# path_namer = 'C:\\Users\\pcosm\\Desktop\\S2-KIVU\\'
# df_namer = glob.glob(os.path.join(path_namer , '*.tif'))
# merge_list = []

# for i in points:
#     merge_mask = i[(i['value'] > minimal)]
#     img_band = df_namer[0].replace(path_namer, '')
#     img_band = img_band.replace('_20m_Mask_NDWI.tif', '')
#     merge_mask.name = img_band
#     df_namer.pop(0)
#     merge_list.append(merge_mask)

In [17]:
for a in points:
    print(a.name)

B11
B12
B2
B3
B4
B5
B6
B7
B8A
B8


In [18]:
for j in points:
    j.to_csv('data_sat/{}.csv'.format(j.name), index=False)

In [19]:
%%time
b2 = pd.read_csv('data_sat/b2.csv')
b3 = pd.read_csv('data_sat/b3.csv')
b4 = pd.read_csv('data_sat/b4.csv')
b5 = pd.read_csv('data_sat/b5.csv')
b6 = pd.read_csv('data_sat/b6.csv')
b7 = pd.read_csv('data_sat/b7.csv')
b8 = pd.read_csv('data_sat/b8.csv')
b8a = pd.read_csv('data_sat/b8a.csv')

Wall time: 8.2 s


In [20]:
kab1 = 1.67 - 3.94 * np.log(b2.value) + 3.78* np.log(b3.value)
sabi = (b8a.value - b4.value) / (b2.value + b3.value)
kivu = (b2.value  - b4.value )/ b3.value
ndci = (b5.value - b4.value )/(b5.value + b4.value)
twobda_1 =  b6.value / b4.value
twobda_2 = b7.value / b4.value
threebda_1 = (b4.value ** -1 - b5.value ** -1) * b6.value
threebda_2 = (b4.value ** -1 - b5.value ** -1) * b7.value
b3b2 =  (b3.value - b2.value) / (b3.value + b2.value)
gb2 = b3.value / b2.value
gr = b3.value / b4.value

In [21]:
kab1.name = 'KAB1_sat'
sabi.name = 'SABI_sat'
kivu.name = 'KIVU_sat'
ndci.name = 'NDCI_sat'
twobda_1.name = '2BDA_1_sat'
twobda_2.name = '2BDA_2_sat'
threebda_1.name = '3BDA_1_sat'
threebda_2.name = '3BDA_2_sat'
b3b2.name = 'B3B2_sat'
gb2.name = 'GB2_sat'
gr.name = 'GR_sat'

In [22]:
# kab1 = pd.concat([kab1, b2.X], axis=1)
# kab1 = pd.concat([kab1, b2.Y], axis=1)

def chl_coords(df1, df2):
    df1 = pd.concat([df1, df2.Latitude], axis= 1)
    df1 = pd.concat([df1, df2.Longitude], axis = 1)
    return df1
kab1 = chl_coords(kab1, b2)
sabi = chl_coords(sabi, b2)
kivu = chl_coords(kivu, b2)
ndci = chl_coords(ndci, b2)
twobda_1 = chl_coords(twobda_1, b2)
twobda_2 = chl_coords(twobda_2, b2)
threebda_1 = chl_coords(threebda_1, b2)
threebda_2 = chl_coords(threebda_2, b2)
b3b2 = chl_coords(b3b2, b2)
gb2 = chl_coords(gb2, b2)
gr = chl_coords(gr, b2)

In [23]:
def renamer(df):
    df = df.rename(columns = {'value': 'df.name'})
    return df
kab1 = renamer(kab1)
sabi = renamer(sabi)
kivu = renamer(kivu)
ndci = renamer(ndci)
twobda_1 = renamer(twobda_1)
twobda_2 = renamer(threebda_2)
threebda_1 = renamer(threebda_1)
threebda_2 = renamer(threebda_2)
b3b2 = renamer(b3b2)
gb2 = renamer(gb2)
gr = renamer(gr)

In [31]:
df_algs = [kab1, sabi, kivu, ndci, twobda_1, twobda_2, threebda_1, threebda_2, b3b2, gb2, gr]
df_names = ['kab1', 'sabi', 'kivu', 'ndci', 'twobda_1', 'twobda_2', 'threebda_1', 'threebda_2', 'b3b2', 'gb2', 'gr']

In [33]:
i = 0
for z in df_algs:
    z.to_csv('data_sat/{}.csv'.format(df_names[i]), index=False)
    i+= 1

In [ ]:
config = configparser.ConfigParser()
config.read_file(open(r'map_config.txt'))
map_config = config.get('map_config', 'map_config')
map_config = ast.literal_eval(map_config)
kepler_map = keplergl.KeplerGl(height=1000, config= map_config)

kepler_map.add_data(data=kivu, name="KIVU")

In [ ]:
#  kepler_map.save_to_html(data={'data_1': kivu}, file_name='kivu.html')

In [ ]:
# kepler_map.add_data(data=kab1, name="KAB1")

In [ ]:
# kepler_map.add_data(data= sabi, name= "SABI")

In [ ]:
kepler_map.add_data(data= ndci, name= "NDCI")

In [ ]:
# kepler_map.add_data(data= twobda_1, name= "2BDA_1")

In [ ]:
kepler_map

In [ ]:
# kepler_map.add_data(data= twobda_2, name= "2BDA_2")

In [ ]:
# kepler_map.add_data(data= threebda_1, name= "3BDA_1")

In [ ]:
# kepler_map.add_data(data= threebda_2, name= "3BDA_MOD")

In [ ]:
# kepler_map.add_data(data= b3b2, name= "B3B1")

In [ ]:
# kepler_map.add_data(data= gb2, name= "GB1")

In [ ]:
# kepler_map.add_data(data= gr, name= "GR")

In [ ]:
# import geopandas
# gdf = geopandas.GeoDataFrame(
#     kab1, geometry=geopandas.points_from_xy(kab1.Longitude, kab1.Latitude))

In [ ]:
# gdf